In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

C:\Users\FEKIYOU\myenvs\signature\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def format_prompt(tokenizer, constat: str, recommandation = None, system_prompt= "Tu es un expert en audit et contrôle interne. Tu génères des recommandations professionnelles et détaillées basées sur les constats d'audit fournis.") -> str:
	"""Format prompt using chat template"""
	messages = [
		{"role": "system", "content": system_prompt},
		{"role": "user", "content": f"Constat d'audit:\n{constat}\n\nGénère des recommandations détaillées pour ce constat."}
	]
	
	if recommandation:
		messages.append({"role": "assistant", "content": recommandation})
	
	return tokenizer.apply_chat_template(
		messages,
		tokenize=False,
						add_generation_prompt=recommandation is None
		)
	
def generate_answer_from_text(model, tokenizer, input_text, device='cuda'):
	prompt = format_prompt(tokenizer, input_text)
	inputs = tokenizer(
		prompt,
		return_tensors="pt",
		truncation=True,
		max_length=2048
	).to(device)
	with torch.no_grad():
		outputs = model.generate(
			**inputs,
			max_new_tokens=1024,
			temperature=0,
			top_p=0.9,
			top_k=50,
			repetition_penalty=1.1,
			pad_token_id=tokenizer.pad_token_id,
			eos_token_id=tokenizer.eos_token_id,
			do_sample=False
		)	
	response = tokenizer.decode(outputs[0], skip_special_tokens=True)
	# Extract only the assistant's response
	if "Génère des recommandations détaillées pour ce constat.\nassistant" in response:
		response = response.split("Génère des recommandations détaillées pour ce constat.\nassistant")[-1].strip()
	return response

In [3]:
device= "cuda"

model_name = "Qwen2.5-1.5B-Instruct-RecoDrafter/"  # Example: "gpt2", "llama-7b", etc.
# Load the pre-trained base model
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 28.84it/s]


In [4]:
# Example usage: generate text from the model
input_text = "- Le comité des risques n’a pas examiné si les prix des produits et services proposés aux clients sont compatibles avec la stratégie en matière de risques, ni proposé de plan d’action en cas d’incompatibilité, conformément aux attributions définies dans l’Actualité Juridique 2015-26.  \n- Le comité des risques n’a pas analysé la compatibilité des incitations de rémunération avec la situation de la Caisse au regard des risques, du capital, de la liquidité et de la probabilité des bénéfices, conformément aux attributions définies dans l’Actualité Juridique 2015-26.  \n- Le comité des nominations n’a pas fixé d’objectif à atteindre en matière de parité hommes-femmes, ni élaboré une politique pour y parvenir, malgré des taux de féminisation inférieurs à 40 %, conformément aux exigences de l’Actualité Juridique 2015-26.  \n- Le comité des nominations n’a pas évalué la structure, la taille, la composition et l’efficacité des travaux du Conseil d’Administration, conformément aux exigences annuelles définies dans l’Actualité Juridique 2015-26.  \n- Le comité des nominations n’a pas vérifié la non domination du Conseil d’Administration par une personne ou un petit groupe de personnes, conformément aux exigences définies dans l’Actualité Juridique 2015-26.  \n- Le comité des nominations n’a pas suivi suffisamment l’évolution des connaissances, expériences et compétences des administrateurs, conformément aux exigences définies dans l’Actualité Juridique 2015-26.  \n- Le comité des nominations a présenté les résultats d’un questionnaire d’auto-évaluation daté du 19/02/2019, mais n’a pas analysé en détail les besoins de formation identifiés (69 % des administrateurs signalant des lacunes en marchés financiers et gestion de la Caisse).  \n- L’assiduité des administrateurs aux formations a été insuffisante en 2018 (13 sur 23), en 2019 (9 sur 23) et en 2020 (11 sur 23), traduisant un suivi inadéquat par le comité des nominations.  \n- Le plan de formation 2020 n’a pas été présenté au comité des nominations, bien que la validation du plan 2021 ait eu lieu en février 2021.  \n- Le comité des nominations a effectué une analyse des profils des candidats au poste d’administrateur et a suivi le taux de féminisation des Conseils d’Administration, mais les informations présentées ne couvrent pas toutes les exigences réglementaires.  \n- Le comité des nominations a présenté des taux de féminisation pour les années 2018, 2019 et 2020, mais ces données n’ont pas été accompagnées d’une analyse approfondie ou d’une communication formelle à l’ACPR.  \n- Le comité des nominations a suivi annuellement le niveau de parité hommes-femmes, mais n’a pas mis en place de mécanismes de suivi ou d’évaluation de l’efficacité de cette démarche.  \n- Le comité des nominations a participé à la validation du plan de formation 2021, mais n’a pas été impliqué dans la présentation ou le suivi du plan 2020, ce qui pourrait affecter la continuité des actions."

In [7]:
print(generate_answer_from_text(model, tokenizer, input_text))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


- 1ère recommandation: Compléter le suivi réalisé en comités des nominations d'administrateurs de la Caisse régionale en veillant à :

	• Rappeler l’obligation de réaliser une auto-évaluation de la connaissance et des capacités à exercer les fonctions d’administrateur ;
	• Présenter un état des lieux semestriel de l'évolution des connaissances, expériences et compétences des Administrateurs de la Caisse régionale permettant de répondre aux différents challenges de notre Gestion.

Livrables attendus :

	• Supports et PV des comités des nominations tenus sur deux occurrences intégrant les éléments demandés par la recommandation
- 2ème recommandation: Renforcer le processus de sélection des nouveaux administrateurs de la Caisse régionale en veillant à :

	• Mettre en place un processus de sélection des Administrateurs intégrant un point de contrôle relatif à la parité homme/femme ;
	• Amender la formalisation de la demande de renouvellement des mandats afin d'intégrer expressément la néce